In [2]:
from PIL import Image
import matplotlib.pyplot as plt
from collections import Counter
import cv2
import numpy as np

In [3]:
def is_blue_tinted(r, g, b, threshold=30, blue_dominance=20):
    return b - ((r + g) / 2) > blue_dominance and b > threshold

def get_average_non_blue_color(pixels, x, y, width, height, radius=3):
    neighbors = []
    for dy in range(-radius, radius + 1):
        for dx in range(-radius, radius + 1):
            nx, ny = x + dx, y + dy
            if 0 <= nx < width and 0 <= ny < height:
                r, g, b = pixels[nx, ny]
                if not is_blue_tinted(r, g, b):
                    neighbors.append((r, g, b))
    if neighbors:
        avg = tuple(int(sum(c) / len(c)) for c in zip(*neighbors))
        return avg
    else:
        return (255, 255, 255)  # fallback if all neighbors are blue

def process_image(img, scale=0.1):
    # Step 1: Downscale for medium quality
    width, height = img.size
    new_size = (int(width * scale), int(height * scale))
    img = img.resize(new_size, Image.LANCZOS)
    pixels = img.load()

    # Step 2: Process blue-tinted regions
    for y in range(img.height):
        for x in range(img.width):
            r, g, b = pixels[x, y]
            if is_blue_tinted(r, g, b):
                avg_color = get_average_non_blue_color(pixels, x, y, img.width, img.height)
                pixels[x, y] = avg_color
    return img


In [7]:
import os
def process_folder(input_dir, output_dir, scale=0.5):
    os.makedirs(output_dir, exist_ok=True)
    supported_exts = ('.png', '.jpg', '.jpeg')

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(supported_exts):
            try:
                img_path = os.path.join(input_dir, filename)
                img = Image.open(img_path).convert("RGB")
                cleaned_img = process_image(img, scale=scale)

                out_path = os.path.join(output_dir, os.path.splitext(filename)[0] + ".png")
                cleaned_img.save(out_path)
                print(f"Saved: {out_path}")
            except Exception as e:
                print(f"Failed to process {filename}: {e}")

# === Example usage ===
input_folder = "../../../dataset/real_life/saihan_bichigten/adiya/raw"
output_folder = "../../../dataset/real_life/saihan_bichigten/adiya/processedv2"
process_folder(input_folder, output_folder, scale=0.3)

Saved: ../../../dataset/real_life/saihan_bichigten/adiya/processedv2\IMG_20250422_111558.png
Saved: ../../../dataset/real_life/saihan_bichigten/adiya/processedv2\IMG_20250422_111611.png
Saved: ../../../dataset/real_life/saihan_bichigten/adiya/processedv2\IMG_20250422_111634.png
Saved: ../../../dataset/real_life/saihan_bichigten/adiya/processedv2\IMG_20250422_111641.png
Saved: ../../../dataset/real_life/saihan_bichigten/adiya/processedv2\IMG_20250422_111652.png
Saved: ../../../dataset/real_life/saihan_bichigten/adiya/processedv2\IMG_20250422_111659.png
Saved: ../../../dataset/real_life/saihan_bichigten/adiya/processedv2\IMG_20250422_111707.png
Saved: ../../../dataset/real_life/saihan_bichigten/adiya/processedv2\IMG_20250422_111718.png
Saved: ../../../dataset/real_life/saihan_bichigten/adiya/processedv2\IMG_20250422_111735.png
Saved: ../../../dataset/real_life/saihan_bichigten/adiya/processedv2\IMG_20250422_111746.png
Saved: ../../../dataset/real_life/saihan_bichigten/adiya/processedv2\I

In [ ]:

def estimate_vertical_skew_and_draw(image: Image.Image, show_debug=True):
    img = np.array(image.convert("RGB"))
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    # Edge detection
    edges = cv2.Canny(gray, 50, 150, apertureSize=3)

    # Probabilistic Hough Line Transform
    lines = cv2.HoughLinesP(edges, rho=1, theta=np.pi/180, threshold=100,
                            minLineLength=100, maxLineGap=10)
    
    angles = []

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
            
            # Consider only nearly vertical lines (angle near 90°)
            if 75 <= abs(angle) <= 105:
                angles.append(angle)
                # if show_debug:
                #     cv2.line(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    if not angles:
        print("No vertical lines detected.")
        return image, 0.0

    median_angle = np.median(angles)
    # print(angles)
    skew_from_vertical = 90 - median_angle 
    # print(f"Detected skew: {skew_from_vertical:.2f}° (to vertical)")

    # Rotate back to vertical
    rotated = image.rotate( skew_from_vertical, resample=Image.BICUBIC, expand=True, fillcolor=(255, 255, 255))

    return rotated




In [ ]:
image_path = '../../../dataset/real_life/saihan_bichigten/adiya/processed/IMG_20250422_111558.png'  # change this to your file path
# image_path = '../../../dataset/real_life/saihan_bichigten/adiya/raw/IMG_20250422_111558.jpg' 
gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)



# Binarize the image (invert to make text=1, background=0)
_, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

In [13]:
img = Image.open(image_path)
debug_img, corrected_img = estimate_vertical_skew_and_draw(img)

debug_img.show()       # Shows image with green detected lines
corrected_img.show() 

[np.float64(-89.11176450476023), np.float64(-88.87669728592458), np.float64(-89.02897806892084), np.float64(-89.15123572844642), np.float64(-87.99746686817312)]
Detected skew: 0.97° (to vertical)
